# Let´s recommend!!

We will load the model that generate candidates and create a function that receives a customer as input and returns a top of N products to be recommended. We will evaluate the results afterwords.

In [2]:
import pandas as pd
import os, sys
import numpy as np
import seaborn as sns
import gc
import warnings
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding
from gensim.models import Word2Vec

In [3]:
path = os.path.join('../../Data/')
data = pd.read_csv(path + 'data_filtered_20190422.csv', sep = ';')
data_processed = pd.read_csv(path + 'data_final_20190525.csv')
data_processed['text'] = data_processed['text'].astype(str)

In [4]:
items_unique = data_processed.item_id.unique()
items_map = {i:val for i,val in enumerate(items_unique)}
items_map_inv = {val:i for i,val in enumerate(items_unique)}
items_map_text = data_processed.set_index('item_id_int').text.to_dict()

In [5]:
idx_customers_map = {i:val for i,val in enumerate(data_processed['customer_id_int'])}
idx_customers_df = pd.DataFrame({'idx': data_processed.index.values, 'customer_id_int': data_processed.customer_id_int})

In [6]:
data.head(3)

,item_id,availability_date,brand,category,name,price
0,A28233506,NaN,Woman Limited El Corte Inglés,"['Moda', 'Mujer', 'Abrigos']",Abrigo masculino con textura de mujer,"{'final': 199, 'currency': 'EUR'}"
1,A29054782,NaN,Woman Limited El Corte Inglés,"['Moda', 'Mujer', 'Abrigos']",Abrigo doble faz de mujer con cinturón a tono,"{'final': 149, 'currency': 'EUR'}"
2,A27354432,NaN,Woman El Corte Inglés,"['Moda', 'Mujer', 'Abrigos']",Abrigo largo de antelina de mujer Woman El Cor...,"{'final': 89.99, 'currency': 'EUR'}"


In [7]:
data_processed.head(3)

,date,item_id,brand,PRICE,customer_id,text,item_age,customer_id_int,item_id_int,brand_id,score,score_original,power_price,power_score,power_item_age,sqrt_price,sqrt_score,sqrt_item_age
0,20190101,A26036172,tintoretto,0.003198,0,vestido mujer flor lazada,0.0,0,0,0,0.001485,4,0.000010,0.000002,0.0,0.056548,0.038538,0.0
1,20190101,A26870590,fórmula joven,0.002260,0,vestido laminado mujer formula joven escote pico,0.0,0,1,1,0.001485,4,0.000005,0.000002,0.0,0.047539,0.038538,0.0
2,20190103,MP_0659870_3014,gabrielle,0.008837,1,abrigo mujer negro avalorios,0.0,1,2,2,0.000000,1,0.000078,0.000000,0.0,0.094005,0.000000,0.0


In [8]:
MAX_NB_WORDS = 30_000 #decided by cumsum wordcount plot (Script 01)
MAX_SEQUENCE_LENGTH = 24 #decided by max words in a product (Script 00)
EMBEDDING_DIM = 100 #Same dim as our W2V embedding

all_text = data_processed['text']
all_text = all_text.drop_duplicates (keep = False)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, )
tokenizer.fit_on_texts(all_text)

data_sequences = tokenizer.texts_to_sequences(data_processed['text'])
data_vec = pad_sequences(data_sequences, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 8487 unique tokens.


In [9]:
model = load_model('../04_Modelado_y_Evaluacion/candidate_generation_20190525')
global graph
graph = tf.get_default_graph() 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


## Predictions

In [10]:
def diversify(arr, diversity, plot = False):
    div = np.log(arr) / diversity
    exp_preds = np.exp(div)
    preds = exp_preds / np.sum(exp_preds)
    if plot:
        plt.figure(figsize = (10, 8));
        plt.subplot(2, 1, 1);
        sns.distplot(arr); plt.title('Original Distribution');
        plt.subplot(2, 1, 2);
        sns.distplot(preds); plt.title(f'Distribution with {diversity} diversity')
    probas = np.random.multinomial(1, preds, 1)
    return probas

In [11]:
def recommend(customer, N = 5):
    try:
        _data = data_processed[data_processed['customer_id_int'] == customer]
        _data_vec = data_vec[_data.index]
        with graph.as_default():
            _pred = model.predict([_data['customer_id_int'], _data['item_id_int'], 
                                 _data['brand_id'], _data['PRICE'],
                                 _data_vec, _data['item_age'], _data['score'],
                                 _data['power_price'], _data['power_score'], _data['power_item_age'],
                                 _data['sqrt_price'], _data['sqrt_score'], _data['sqrt_item_age']],
                                 verbose = 1)
        _pred = pd.DataFrame(_pred)
        _pred['customer_id_int'] = customer
        _pred = _pred.groupby(['customer_id_int']).max()
        del _pred.index.name
        
    #########################################################
        #_pred = diversify(_pred.values.reshape(_pred.shape[1]), diversity = 0.7, plot = False)
        #print(_pred)
        print('\n' + '=='*30 + '\n')
        print(f'==> Top {N} Recommended items to Customer {customer}: ')
        print(f'\nThe customer {customer} has bought this items: ')
        print('\n' + '=='*30 + '\n')
        interacted_items = data_processed[['text', 'score_original']][data_processed['customer_id_int'] == customer].groupby('text')\
                            .sum().reset_index().sort_values(['score_original'], ascending = False)
        print('\n'.join([str(i+1) + str(' - ') + str(x) for i, x in enumerate(interacted_items['text'].values[0:20])]))
        top = _pred.values.reshape(_pred.shape[1]).argsort()[-N:][::-1] #items positions
        print('\n====================== IDs DE PRODUCTOS RECOMENDADOS ==============')
        print([items_map[item] for item in top])
        print ("\n===================== PRODUCTOS RECOMENDADOS =====================")
        print('\n'.join([str(i+1) + str(' - ') + str(items_map_text[x]) for i, x in enumerate(top)]))
        print ("==================================================================")
    except:
        print(f'\nThe customer {customer} does not exist')

In [17]:
def recommend_1(customer  , N = 5):
    text = ''
    try:
        _data = data_processed[data_processed['customer_id_int'] == customer]
        _data_vec = data_vec[_data.index]
        _pred = model.predict([_data['customer_id_int'], _data['item_id_int'], 
                             _data['brand_id'], _data['PRICE'],
                             _data_vec, _data['item_age'], _data['score'],
                             _data['power_price'], _data['power_score'], _data['power_item_age'],
                             _data['sqrt_price'], _data['sqrt_score'], _data['sqrt_item_age']],
                             batch_size = 1, verbose = 0)
        _pred = pd.DataFrame(_pred)
        _pred['customer_id_int'] = customer
        _pred = _pred.groupby(['customer_id_int']).max()
        del _pred.index.name
        
    #########################################################
        #_pred = diversify(_pred.values.reshape(_pred.shape[1]), diversity = 0.7, plot = False)
        #print(_pred)
        text += '\n' + '=='*30 + '\n'
        text += '==> Top ' + str(N) + 'Recommended items to Customer ' + str(customer) + ': '
        text += '\nThe customer ' + str(customer) + 'has bought this items: '
        text += '\n' + '=='*30 + '\n'

        interacted_items = data_processed[['text', 'score_original']][data_processed['customer_id_int'] == customer].groupby('text')                            .sum().reset_index().sort_values(['score_original'], ascending = False)
        text += '\n'.join([str(i+1) + str(' - ') + str(x) for i, x in enumerate(interacted_items['text'].values[0:20])])
        top = _pred.values.reshape(_pred.shape[1]).argsort()[-N:][::-1] #items positions
        text += '\n====================== IDs DE PRODUCTOS RECOMENDADOS =============='
        text += ''.join([str(items_map[item]) for item in top])
        text += "\n===================== PRODUCTOS RECOMENDADOS ====================="
        text += '\n'.join([str(i+1) + str(' - ') + str(items_map_text[x]) for i, x in enumerate(top)])
        text += "=================================================================="
    except Exception as e:
        print(f'Exception: {e}')
        print(f'\nThe customer {customer} does not exist')
        
    return text

In [18]:
CUSTOMER = 1 #128 #270
recommend_1(customer = CUSTOMER, N = 10)

aaaaa


'\n============================================================\n==> Top 10Recommended items to Customer 1: \nThe customer 1has bought this items: \n============================================================\n1 - reloj mujer negro\n2 - jersey manga larga mujer talla grande bajo plisado\n3 - bandolera mujer ginny piel granulada borla\n4 - pulsera cuero metal dorado\n5 - campana telescopica cnl velocidad\n6 - reloj gps pulsometro vantage talla\n7 - jersey mujer talla grande plus escote pico estampado paisley\n8 - mercedita mujer charol color topo\n9 - mesa auxiliar cristal\n10 - mesa auxiliar top\n11 - mesita noche cajon mollis\n12 - pantalon fluido mujer talla grande plus cintura elastica\n13 - prensa ajo mejor precio corte_ingles\n14 - pulsera plata corte_ingles dorada triangulo\n15 - abrigo bicolor mujer talla grande plus cuello alto\n16 - jersey gris mujer talla grande plus escote bordado\n17 - reloj inteligente smartwatch galaxy watch milimetro bt plata\n18 - reloj mujer brazalete

---